# Forest Fire Detection with transfer learning using VGG19

In [14]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras.models import load_model
from keras.applications import VGG19
from keras import layers
from keras import Model
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
train_dir = "data/Training"
test_dir = "data/Test"

VGG_SIZE = 224

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                validation_split=0.25)

test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                subset='training',
                                                class_mode='binary')

val_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                subset='validation',
                                                class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE),
                                                color_mode='rgb',
                                                batch_size=20,
                                                class_mode='binary')

Found 29547 images belonging to 2 classes.
Found 9848 images belonging to 2 classes.
Found 8617 images belonging to 2 classes.


In [18]:
transfer_model = VGG19(include_top=True,
                        weights="imagenet",
                        classes=1000)
input = transfer_model.inputs

In [19]:
x = transfer_model.get_layer("block5_pool").output
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(2048, activation='relu')(x)
x = layers.Dropout(0.15)(x)
x = layers.Dense(1024, activation="relu")(x)
output_layer = layers.Dense(1, activation='sigmoid')(x)

classifier = Model(input, output_layer)

for layer in classifier.layers[:-4]:
    layer.trainable = False

classifier.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [20]:
classifier.compile(loss="binary_crossentropy",
                optimizer=optimizers.Adam(lr=0.001),
                metrics=["accuracy"])

In [21]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    ModelCheckpoint(filepath='models/vgg19_model.h5', save_best_only=True),
]

history = classifier.fit(train_generator, 
                        epochs=5, 
                        batch_size=40,
                        validation_data=test_generator,
                        callbacks=callbacks)

Epoch 1/5
1478/1478 [==============================] - 6210s 4s/step - loss: 0.1539 - accuracy: 0.9566 - val_loss: 0.0989 - val_accuracy: 0.9563
Epoch 2/5
1478/1478 [==============================] - 5892s 4s/step - loss: 0.0768 - accuracy: 0.9712 - val_loss: 0.0596 - val_accuracy: 0.9734
Epoch 3/5
1478/1478 [==============================] - 6097s 4s/step - loss: 0.0615 - accuracy: 0.9772 - val_loss: 0.1069 - val_accuracy: 0.9566
Epoch 4/5
1478/1478 [==============================] - 6114s 4s/step - loss: 0.0589 - accuracy: 0.9786 - val_loss: 0.1684 - val_accuracy: 0.9418
Epoch 5/5
1478/1478 [==============================] - 6031s 4s/step - loss: 0.0510 - accuracy: 0.9807 - val_loss: 0.1488 - val_accuracy: 0.9409


In [22]:
#classifier.save("models/vgg19_model.h5")

In [23]:
result = classifier.evaluate(test_generator)
print(f"Test loss: {result[0]}, Test accuracy: {result[1]}")

431/431 [==============================] - 1247s 3s/step - loss: 1.2371 - accuracy: 0.7248
Test loss: 1.2370550632476807, Test accuracy: 0.7248462438583374


In [24]:
from keras import models

model = models.load_model("models/vgg19_model_or.h5")
result = model.evaluate(test_generator)
print(f"Test loss: {result[0]}, Test accuracy: {result[1]}")

431/431 [==============================] - 1239s 3s/step - loss: 3.2376 - accuracy: 0.6402
Test loss: 3.2375502586364746, Test accuracy: 0.6402460336685181
